# Memory leak on exception

This notebook demonstrates a temporary memory leak that happens when an exception occurs 

In [1]:
import numpy as np
import gc, os, sys, time, psutil

In [2]:
def consume_cpu_ram(n): return np.ones((n, n))
def consume_cpu_ram_128mb():  return consume_cpu_ram(2**12)

process = psutil.Process(os.getpid())
def cpu_ram_used():  return process.memory_info().rss

In [3]:
def fail():
    x = consume_cpu_ram_128mb()
    raise ValueError("Ouch")

In [4]:
before = cpu_ram_used()
fail()

ValueError: Ouch

In [5]:
_ = gc.collect()
after = cpu_ram_used()
diff = int((after-before)/2**20)
# Without the leak the difference should be ~0
# With the leak, the locals() are tied up in the tb and aren't released, so expecting ~128MB
assert diff < 2, f"got leak of {diff} MB"

AssertionError: got leak of 128 MB

In [6]:
# force ipython to reset its %tb
raise ValueError("Reset")

ValueError: Reset

In [7]:
_ = gc.collect()
after = cpu_ram_used()
diff = int((after-before)/2**20)
# 2nd exception resets locals tied up in tb, and they now can be released
# so expecting difference close to 0MB
assert diff < 2, f"got Difference {diff} MB"